In [ ]:
import data.sf
print(dir(data.sf))  # Shows all available functions

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [ ]:
from data.data_preprocessing import lo

ImportError: cannot import name 'load_credit_data' from 'data.data_preprocessing' (c:\Users\banji\creditCard_default_risk\src\data\data_preprocessing.py)

In [12]:
from data.data_loading import load_credit_data
df =  load_credit_data()
print(df.head())

   credit_amount  gender  education  marital_status  age  sept_delay  \
0          20000       2          2               1   24           2   
1         120000       2          2               2   26          -1   
2          90000       2          2               2   34           0   
3          50000       2          2               1   37           0   
4          50000       1          2               1   57          -1   

   august_delay  july_delay  june_delay  may_delay  ...  june_bill  may_bill  \
0             2          -1          -1         -2  ...          0         0   
1             2           0           0          0  ...       3272      3455   
2             0           0           0          0  ...      14331     14948   
3             0           0           0          0  ...      28314     28959   
4             0          -1           0          0  ...      20940     19146   

   april_bill  sept_payment  august_payment  july_payment  june_payment  \
0          

In [5]:
# Create copy for Exploratory Data Analysis
df_eda = df.copy()
df_eda.head()

# Check for null and duplicate entries
print("===== Null entries =====")
print(df_eda.isna().sum())
print()
print("===== Duplicate entries =====")
print(df_eda.duplicated().sum())


===== Null entries =====
credit_amount     0
gender            0
education         0
marital_status    0
age               0
sept_delay        0
august_delay      0
july_delay        0
june_delay        0
may_delay         0
april_delay       0
sept_bill         0
august_bill       0
july_bill         0
june_bill         0
may_bill          0
april_bill        0
sept_payment      0
august_payment    0
july_payment      0
june_payment      0
may_payment       0
april_payment     0
Y                 0
dtype: int64

===== Duplicate entries =====
35


In [ ]:
from data.sf import split_columns

ImportError: cannot import name 'split_columns' from 'data.data_preprocessing' (c:\Users\banji\creditCard_default_risk\src\data\data_preprocessing.py)

In [ ]:
from data.sf import split_columns
#split_columns(df=df_eda)

ImportError: cannot import name 'split_columns' from 'data.data_preprocessing' (c:\Users\banji\creditCard_default_risk\src\data\data_preprocessing.py)

In [ ]:

# Creation of categorical map for categorical data for easy understanding of the dataset
gender_map = {
    1: 'male',
    2: 'female'
}

education_map = {
    0: 'others',
    1: 'graduate school',
    2: 'university',
    3: 'high school',
    4: 'others',
    5: 'others',
    6: 'others'
}

marital_map = {
    0: 'others',
    1: 'married',
    2: 'single',
    3: 'divorce'
               }

target_map = {
    0: 'non-default',
    1: 'default'
    }

# Payment Delay Map
delay_map = {
  -2: 'no usage',
  -1: 'paid full',
   0: 'revolving',
   1: '1m delay',
   2: '2m delay',
   3: '3m delay',
   4: '4m delay',
   5: '5m delay',
   6: '6m delay',
   7: '7m delay',
   8: '8m delay',
   9: '9m+ delay'
   }

# function to group ages into decades
def convert_age(age):
    """
    Converts age into decades for easy exploration.
    
    Parameters
    ----------
    age : int
        Age of consumer.


    Returns
    -------
    decade : str
        Age converted into relevant decade.
    
    """
    
    # divide age by 10 to determine the decade
    age = age//10
    decade = f'{age*10}s'
    return decade
print("===== Five rows for categorical data (before mapping) =====")
print(df_eda[categorical_data].head())

# application of the categorical map to all the columns
df_eda['gender'] = df_eda['gender'].map(gender_map )
df_eda['education'] = df_eda['education'].map(education_map)
df_eda['marital_status'] = df_eda['marital_status'].map(marital_map)
df_eda['Y'] = df_eda['Y'].map(target_map)
df_eda['age'] = df_eda['age'].map(convert_age)



ImportError: cannot import name 'split_columns' from 'data.data_preprocessing' (c:\Users\banji\creditCard_default_risk\src\data\data_preprocessing.py)

In [ ]:
# plot to show credit column before and after transformation
fig, ax = plt.subplots(1,2, figsize = (7,4.5))

# First Plot
df['credit_amount'].plot(kind = 'hist', bins = 30, ax= ax[0])
ax[0].ticklabel_format(style='plain', axis='x')
ax[0].set_title('Original Credit Amount')

# second plot 
np.sqrt(df['credit_amount']+1).plot(kind = 'hist', bins = 30, ax = ax[1]) # looks a bit gaussian
#np.log(X_train['credit_amount']).plot(kind = 'hist', bins = 30, ax = ax[1])
ax[1].set_title('Transformed Credit Amount')
plt.tight_layout()
plt.savefig(fname = 'transf', 
        dpi=300, 
        bbox_inches='tight',    
        pad_inches=0.1,         
        facecolor='white')
plt.show()
delay_cols = [f'{month}_delay' for month in months]

df['payment_consistency'] = df[['sept_payment', 
                                'august_payment', 
                                'july_payment', 
                                'june_payment', 
                                'may_payment', 
                                'april_payment']].std(axis=1)


df['total_delays'] = df[delay_cols].sum(axis=1)